In [12]:
import pandas as pd

from tqdm.notebook import tqdm

In [35]:
test_df = pd.read_csv('test.csv')

In [36]:
test_df.dropna(inplace=True)

In [37]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_CHECKPOINT = './google/mt5-base-finetuned-asr_correction-hindi'
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

C:\Users\Hanish\AppData\Roaming\Python\Python311\site-packages\transformers\convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [38]:
def correct(text, model=model, tokenizer=tokenizer):
    input_ids = tokenizer(text, return_tensors="pt")["input_ids"]
    output = model.generate(input_ids)
    translated_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    return translated_text

In [41]:
test_df

,og_transcription,ms_asr_transcription
0,मेरे बेटे के होंठ में छेद हो गया था और वह सूज ...,मेरे बेटे के होम्स में छेद हो गया था और वो सूझ...
1,मेरी पीठ के निचले हिस्से की मांसपेशियाँ दर्द क...,मेरी पीठ के निचले हिस्से की मांसपेशियां दर्द क...
2,मेरी पीठ की मांसपेशियों में दर्द है,मेरी पीठ के मांसपेशियों में दर्द है।
3,मेरे बाएं पैर की मांसपेशियों में दर्द है,मेरे भाई पै की मांसपेशियों में दर्द है।
4,फुटबॉल खेलने के कारण मेरी उंगली कट गई है और मु...,फुटबॉल खेलने के कारण मेरी उगली कट गई है और मुझ...
...,...,...
180,मैं अपने घुटनों के दर्द के कारण व्यायाम नहीं क...,मैं अपने घुटनों के दर्द के कारण व्यायाम नहीं क...
181,मुझे ठीक से सुनाई नहीं देता मेरे कान में समस्य...,मुझे ठीक से सुनाई नहीं देता मेरे कान में समस्य...
182,बालों का भयानक रूप से झड़ना,बालों का भयानक रूप से झना।
183,मेरे कान में मुझे बहुत दर्द हो रहा है,मेरे कान में मुझे बहुत दर्द हो रहा है'।


In [ ]:
final_df = []

for og, ms in zip(test_df['og_transcription'], test_df['ms_asr_transcription']):
    final_df.append([og, ms, correct(ms)])

In [59]:
final_df = pd.DataFrame(final_df, columns=['og_transcript', 'ms_asr_transcription', 'corrected_transcription'])

In [60]:
import re

def remove_characters(line):
    return re.sub('[.,।]', '', line)

def find_diff_words(str1, str2):
    str1 =remove_characters(str1)
    str2 =remove_characters(str2)

    words1 = str1.split()
    words2 = str2.split()
    diff_words = []
    for i in range(min((len(words1), len(words2)))):
        if words1[i].lower() != words2[i].lower():
            # print(words1[i], words2[i])
            diffs = [i, words1[i], words2[i]]
            diff_words.append(diffs)
    return diff_words

In [61]:
differnces_og_ms = []
for lin1, lin2 in zip(final_df['ms_asr_transcription'], final_df['og_transcript']):
    differnces_og_ms.append(find_diff_words(lin1, lin2))

final_df['differnces_og_ms'] = differnces_og_ms

In [62]:
differnces_og_corr = []
for lin1, lin2 in zip(final_df['corrected_transcription'], final_df['og_transcript']):
    differnces_og_corr.append(find_diff_words(lin1, lin2))

final_df['differnces_og_corr'] = differnces_og_corr

In [63]:
differnces_ms_corr = []
for lin1, lin2 in zip(final_df['corrected_transcription'], final_df['ms_asr_transcription']):
    differnces_ms_corr.append(find_diff_words(lin1, lin2))

final_df['differnces_ms_corr'] = differnces_ms_corr

In [65]:
final_df.to_csv('test_hindi.csv', index=False)

In [64]:
final_df

,og_transcript,ms_asr_transcription,corrected_transcription,differnces_og_ms,differnces_og_corr,differnces_ms_corr
0,मेरे बेटे के होंठ में छेद हो गया था और वह सूज ...,मेरे बेटे के होम्स में छेद हो गया था और वो सूझ...,मेरे बेटे के होम्स में छेद हो गया था और उसके हो,"[[3, होम्स, होंठ], [10, वो, वह], [11, सूझ, सूज...","[[3, होम्स, होंठ], [10, उसके, वह], [11, हो, सूज]]","[[10, उसके, वो], [11, हो, सूझ]]"
1,मेरी पीठ के निचले हिस्से की मांसपेशियाँ दर्द क...,मेरी पीठ के निचले हिस्से की मांसपेशियां दर्द क...,मेरी पीठ के निचले हिस्से की मांसपेशियां दर्द कर,"[[6, मांसपेशियां, मांसपेशियाँ]]","[[6, मांसपेशियां, मांसपेशियाँ]]",[]
2,मेरी पीठ की मांसपेशियों में दर्द है,मेरी पीठ के मांसपेशियों में दर्द है।,मेरी पीठ के मांसपेशियों में दर्द है,"[[2, के, की]]","[[2, के, की]]",[]
3,मेरे बाएं पैर की मांसपेशियों में दर्द है,मेरे भाई पै की मांसपेशियों में दर्द है।,मेरे भाई पै की मांसपेशियों में दर्द है,"[[1, भाई, बाएं], [2, पै, पैर]]","[[1, भाई, बाएं], [2, पै, पैर]]",[]
4,फुटबॉल खेलने के कारण मेरी उंगली कट गई है और मु...,फुटबॉल खेलने के कारण मेरी उगली कट गई है और मुझ...,फुटबॉल खेलने के कारण मेरी उगली कट गई है,"[[5, उगली, उंगली], [14, न, लगानी], [15, गानी, ...","[[5, उगली, उंगली]]",[]
...,...,...,...,...,...,...
161,मेरे मध्य कान में संक्रमण के कारण समस्या हो गई है,मेरे मध्यकान में संक्रमण के कारण समस्या हो गई है।,मेरे मध्यकान में संक्रमण के कारण समस्या हो गई है,"[[1, मध्यकान, मध्य], [2, में, कान], [3, संक्रम...","[[1, मध्यकान, मध्य], [2, में, कान], [3, संक्रम...",[]
162,मेरे अंगूठे और तर्जनी के बीच की त्वचा में चोट ...,मेरे अंगू के और तर्जनी के बीच की त्वचा में चोट...,मेरे अंगू के और तर्जनी के बीच की त्वचा में,"[[1, अंगू, अंगूठे], [2, के, और], [3, और, तर्जन...","[[1, अंगू, अंगूठे], [2, के, और], [3, और, तर्जन...",[]
163,जब मैं अपने बालों को ब्रश करता हूं तो देखता हू...,जब मैं अपने बालों को वृष करता हूँ तो देखता हूँ...,जब मैं अपने बालों को वृष करता हूं तो देखता ह,"[[5, वृष, ब्रश], [7, हूँ, हूं], [10, हूँ, हूं]...","[[5, वृष, ब्रश], [10, ह, हूं]]","[[7, हूं, हूँ], [10, ह, हूँ]]"
164,चलते समय घुटने में दर्द महसूस होता है,चलते समय घुटने में दर्द महसूस होता है।,मुझते समय घुटने में दर्द महसूस होता है,[],"[[0, मुझते, चलते]]","[[0, मुझते, चलते]]"
